In [ ]:
# stdlib
import os
from os import environ as env
from time import sleep

# syft absolute
import syft as sy
from syft.util.test_helpers.email_helpers import SENDER
from syft.util.test_helpers.email_helpers import get_email_server

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin@bigquery.org", "bqpw"

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ADMIN_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ADMIN_PW

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
email_server, smtp_server = get_email_server()

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
high_client.api.services.settings.enable_notifications(
    email_sender=SENDER,
    email_server="localhost",
    email_port="9025",
)

# Check default email instant deliver

In [ ]:
@sy.syft_function_single_use()
def test_syft_function():  # noqa: F811
    print("Testing syft function request email notification")
    return True


# Create a project
project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[high_client],
)
project.create_code_request(test_syft_function, high_client)

In [ ]:
assert (
    "A new request has been submitted and requires your attention."
    in email_server.load_emails()[ADMIN_EMAIL][-1].email_content
)

## Batch Notifications

In [ ]:
current_admin_email_cache_len = len(email_server.load_emails()[ADMIN_EMAIL])

In [ ]:
# syft absolute
from syft.service.notifier.notifier_enums import EMAIL_TYPES
from syft.service.notifier.notifier_enums import NOTIFICATION_FREQUENCY

high_client.settings.batch_notifications(
    EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.DAILY, start_time="6:00 pm"
)

In [ ]:
@sy.syft_function_single_use()
def test_syft_function_data_upload():  # noqa: F811
    print("Testing data upload request notification email")
    return True


project.create_code_request(test_syft_function_data_upload, high_client)


@sy.syft_function_single_use()
def test_syft_function_model_training():  # noqa: F811
    print("Testing model training request notification email")
    return True


project.create_code_request(test_syft_function_model_training, high_client)


@sy.syft_function_single_use()
def test_syft_function_data_analysis():  # noqa: F811
    print("Testing data analysis request notification email")
    return True


project.create_code_request(test_syft_function_data_analysis, high_client)


@sy.syft_function_single_use()
def test_syft_function_data_cleaning():  # noqa: F811
    print("Testing data cleaning request notification email")
    return True


project.create_code_request(test_syft_function_data_cleaning, high_client)


@sy.syft_function_single_use()
def test_syft_function_report_generation():  # noqa: F811
    print("Testing report generation request notification email")
    return True


project.create_code_request(test_syft_function_report_generation, high_client)

In [ ]:
assert current_admin_email_cache_len == len(email_server.load_emails()[ADMIN_EMAIL])

In [ ]:
high_client.settings.batch_notifications(
    EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.INSTANT
)

In [ ]:
sleep(15)

In [ ]:
assert current_admin_email_cache_len + 1 == len(email_server.load_emails()[ADMIN_EMAIL])

In [ ]:
assert (
    "Batched Requests Notification"
    in email_server.load_emails()[ADMIN_EMAIL][-1].email_content
)

## Test Setting batch notifications without passing start time

In [ ]:
with sy.raises(sy.SyftException, show=True):
    high_client.settings.batch_notifications(
        EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.DAILY
    )

## Test setting batch notifications with bad formatted start time

In [ ]:
with sy.raises(sy.SyftException, show=True):
    high_client.settings.batch_notifications(
        EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.DAILY, "6 pm"
    )

In [ ]:
smtp_server.stop()

In [ ]:
server.land()